<a href="https://colab.research.google.com/github/jithesh9539/Approximate_Nearest-Neighbor_Search/blob/main/ANN_Algotithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import pickle

     |████████████████████████████████| 310 kB 31.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705348 sha256=8148d6964ebf6c5b41703e0d81f705904f6937da47b592f47cf653f46e406060
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [26]:
import numpy as np

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [27]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [31]:
train = movielens['train']
test = movielens['test']
item_vectors = movielens['item_features'] * model.item_embeddings

In [21]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.59, test 0.10.
AUC: train 0.90, test 0.86.


In [22]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.61, test 0.11.
AUC: train 0.94, test 0.90.


In [32]:
with open('movielens.pickle', 'wb') as f:
    pickle.dump({"name": movielens['item_feature_labels'], "vector": item_vectors}, f)

In [33]:
def load_data():
    with open('movielens.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.47669363,  0.76664734,  0.5728357 , ..., -0.98119307,
          0.18483748,  0.31369993],
        [ 0.05845284,  0.17278723,  0.5082774 , ..., -0.53795415,
          0.2532802 , -0.9027846 ],
        [ 0.19099018,  0.30912766,  0.22659147, ..., -0.33714873,
          0.09478349, -0.00221124],
        ...,
        [ 0.46408868, -0.47072574, -0.57541287, ...,  0.51241237,
          0.29486287,  0.07820068],
        [ 0.20238926, -0.31854233, -0.1574961 , ...,  0.3242837 ,
          0.16420357, -0.0982646 ],
        [ 0.39446005, -0.3929522 , -0.31282395, ...,  0.19513677,
          0.10329727,  0.00908695]], dtype=float32)}

In [34]:
!pip install faiss-gpu
import faiss

     |████████████████████████████████| 89.7 MB 13 kB/s 


**Locality_Sensitive_Hashing**

In [35]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [36]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [38]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mr. Smith Goes to Washington (1939)
* Gone with the Wind (1939)
* Lion King, The (1994)
* Homeward Bound: The Incredible Journey (1993)
* Bedknobs and Broomsticks (1971)
* Wrong Trousers, The (1993)
* Pinocchio (1940)
* Aristocats, The (1970)
* Snow White and the Seven Dwarfs (1937)


**Trees** and Graphs using Annoy

In [39]:
class TreesIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [42]:
index = TreesIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [41]:
!pip install annoy
import annoy

     |████████████████████████████████| 646 kB 41.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391683 sha256=661e5f39568d5ccf95e15ca3de4a60f675c46ab0a0be9d3e428b5fe9af5aec64
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [43]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
similar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Mr. Smith Goes to Washington (1939)
* Gone with the Wind (1939)
* Lion King, The (1994)
* Homeward Bound: The Incredible Journey (1993)
* Bedknobs and Broomsticks (1971)
* Wrong Trousers, The (1993)
* Pinocchio (1940)
* Aristocats, The (1970)
* Snow White and the Seven Dwarfs (1937)


In [44]:
!pip install nmslib
import nmslib

     |████████████████████████████████| 13.5 MB 37.0 MB/s 
     |████████████████████████████████| 188 kB 77.5 MB/s 


**HNSW: Hierarchical Navigable Small World Algorithm**

In [45]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [46]:
index = HNSWIndex(data["vector"], data["name"])
index.build()

In [47]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar stack to {movie_name} are:\n* {simlar_movie_questions}")

The most similar stack to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Sword in the Stone, The (1963)
* Jurassic Park (1993)
* Alice in Wonderland (1951)
* Cinderella (1950)
* Snow White and the Seven Dwarfs (1937)
* Aladdin (1992)
* Fantasia (1940)
* Star Trek: The Wrath of Khan (1982)
* True Lies (1994)


**Product** **Quantization**

In [48]:
class ProductIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [49]:
index = ProductIndex(data["vector"], data["name"])
index.build()

In [51]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movie to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movie to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Sword in the Stone, The (1963)',
 'Alice in Wonderland (1951)',
 'Cape Fear (1991)',
 'Three Musketeers, The (1993)',
 'Maverick (1994)',
 'Up in Smoke (1978)',
 'Heavy Metal (1981)',
 'Casper (1995)',
 'Mask, The (1994)']

**Exhaustive** **Search**

In [52]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [53]:
index = ExhaustiveIndex(data["vector"], data["name"])
index.build()

In [54]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Sword in the Stone, The (1963)
* Alice in Wonderland (1951)
* Cinderella (1950)
* Snow White and the Seven Dwarfs (1937)
* Fantasia (1940)
* Cape Fear (1991)
* Heathers (1989)
* Aladdin (1992)
* Up in Smoke (1978)
